# Polars quickstart
 
To help you get started this notebook introduces some of the key concepts that make Polars a powerful data analysis tool.

The key concepts we meet are:
- fast flexible analysis with the Expression API in Polars
- easy parallel computations
- automatic query optimisation in lazy mode
- streaming to work with larger-than-memory datasets in Polars

## Stay in touch
I post a lot of material about Polars on social media and my blog. Stay in touch by
- connecting with me on LinkedIn https://www.linkedin.com/in/liam-brannigan-9080b214a/
- following me on twitter https://twitter.com/braaannigan
- check out my blog posts https://www.rhosignal.com/
- see my youtube channel https://www.youtube.com/channel/UC-J3uR0g7CxCSnx0YFE6R_g/

Send a message to say hi if you are coming from the course! 

## Importing Polars
We begin by importing polars as `pl`. Following this convention will allow you to work with examples from the official documentation

In [ ]:
import polars as pl

## Setting configuration options
We want to control how many rows of a `DataFrame` are printed out to the screen. Polars allows us to control configuration using options using methods in the `pl.Config` namespace.

In this notebook we want Polars to print 6 rows of `DataFrame` so we use `pl.Config.set_tbl_rows`

In [ ]:
pl.Config.set_tbl_rows(6)

You can see the full range of configuration options here: https://pola-rs.github.io/polars/py-polars/html/reference/config.html

In the course we see how to apply the right configuration options in a range of contexts.

## Input data
Polars can read from a wide range of data formats including CSV, Parquet, Arrow, JSON, Excel and database connections. We cover all of these in the course.

For this introduction we use a CSV with the Titanic passenger dataset. This dataset gives details of all the passengers on the Titanic and whether they survived.

We begin by setting the path to this CSV

In [ ]:
csv_file = "../data/titanic.csv"

We read the CSV into a Polars `DataFrame` with the `read_csv` function. 

We then call `head` to print out the first few rows of the `DataFrame`

In [ ]:
df = pl.read_csv(csv_file)
df.head(3)

Each row of the `DataFrame` has details about a passenger on the Titanic including the class they travelled in (`Pclass`), their name (`Name`) and `Age`.

Alternatively we can use `glimpse` to see the first data points arranged vertically. I use this regularly for dataframes with a lot of columns

In [ ]:
print(df.glimpse())

## Expressions
You can use square brackets to select rows and columns in Polars...

In [ ]:
df[:3,["Pclass","Name","Age"]]

...but using this square bracket approach means that you don't get all the benefits of parallelisation and query optimisation.

> We learn more about square bracket indexing in Section 3 of the course.

To really take advantage of Polars we use the Expression API.

### Selecting and transforming columns with the Expression API

We see a simple example of the Expression API here where we select the `Pclass`, `Name` and `Age` columns inside a `select` statement (we learn much more about a `select` statement in Section 3)

In [ ]:
(
    df
    .select(
        [
            pl.col("Pclass"),
            pl.col("Name"),
            pl.col("Age"),
        ]
    )
)

In the Expression API we use `pl.col` to refer to a column.

We would like the strings in the `Name` column to be printed wider. We can do this with

In [ ]:
pl.Config.set_fmt_str_lengths(100)

> We learn more about the `pl.Config` namespace for configuring how Polars looks and behaves in a lecture later in this Section.

### What is an expression?

An expression is a function that takes a `Series` (or column in a `DataFrame`) in and returns `Series` (or column in a `DataFrame`). 

Expressions are the core building blocks of data transformations and include:
- the identity expression where the output is the same as the input
- arithmetic where we add/multiply/etc all elements of a `Series`
- rounding off all elements of a `Series`
- converting all strings in a `Series` to uppercase
- extracting the date from all elements of a datetime `Series`
- and so on

In this example we select the same three columns, but this time we:
- convert the names to lowercase and
- round off the age to 2 decimal places

In [ ]:
(
    df
    .select(
        [
            # Identity expression
            pl.col("Pclass"),
            # Names to lowercase
            pl.col("Name").str.to_lowercase(),
            # Round the ages
            pl.col("Age").round(2)
        ]
    )
)

When we have multiple expressions like this Polars runs them in parallel.

Expressions can also return a shorter `Series` such as `head` to return the first rows or aggregating expressions such as `mean` to get the average of the values in a `Series`. Expressions can also return a longer `Series` such as `explode` that converts a list `Series` to individual rows.

> We learn much more about expressions in Section 3 of the course.

### Method chaining and code formatting
In the cell above the code is wrapped in parantheses `()`. In Python (rather than Polars in particular) when we wrap code in parantheses we can call a new method - in this case `select` - on a new line.

In Polars we often build queries in multiple steps with multiple calls to new methods. I find it is much easier to read a series of queries if each method starts on a new line so I will generally wrap code blocks in paranetheses.

### Expression chaining

As well as chaining methods we can chain expressions together to do more transformations in a single step. 

In this example we return three columns:
- the original `Name` columns
- the `Name` column split into a list of words
- the count of the number of words when the `Name` column split into a list of words

Column names in a Polars `DataFrame` are always strings and must be unique. We use the `alias` method at the end of the second and third expressions so we do not end up with multiple columns called `Name`

In [ ]:
(
    df
    .select(
        [
            # Get the Name column without changes
            pl.col("Name"),
            # Take the Name column and split it into a list of separate words
            pl.col("Name").str.split(" ").alias("Name_split"),
            # Take the Name column, split it into a list of separate words and count the number of words
            pl.col("Name").str.split(" ").list.len().alias("Name_word_count"),
        ]
    )
)

We look at expressions in detail throughout the course to find the right expression for many different scenarios.

Expressions can seem verbose, but they also allow us to select groups of columns in one go. For example, to select all the integer columns we can use `pl.

In [ ]:
(
    df
    .select(
        pl.col(pl.INTEGER_DTYPES)
    )
    .head(3)
)

> We meet other ways to quickly select multiple columns in Section 3.

### Filtering a `DataFrame` with the Expression API

We filter a `DataFrame` by applying a condition to an expression.

In this example we find all the passengers over 70 years of age

In [ ]:
(
    df
    .filter(
        pl.col("Age") > 70
    )
)

We are not limited to using the Expression API for these operations. The Expression API is at the heart of all data transformations in Polars as we see below.

> We learn more about applying filter conditions in Section 2 of the course.

## Analytics
Polars has a wide range of functionality for analysing data. In the course we look at a wider range of analytic methods and how we can use expressions to write more complicated analysis in a concise way.

We begin by getting an overview of the `DataFrame` with `describe`

In [ ]:
df.describe()

The output of `describe` shows us how many records there are, how many `null` values and some key statistics. The `null_count` has helped me identify emerging data quality issues in my machine learning pipelines.

### Value counts on a column
We use `value_counts` to count occurences of values in a column.

In this example we count how many passengers there are in each class with `value_counts`

In [ ]:
df["Pclass"].value_counts()

### Groupby and aggregations
Polars has a fast parallel algorithm for `group_by` operations. 

Here we first group by the `Survived` and the `Pclass` columns. We then aggregate in `agg` by counting the number of passengers in each group

In [ ]:
(
    df
    .group_by(["Survived","Pclass"])
    .agg(
        pl.col("PassengerId").count().alias("count")
    )
)

We use the Expression API to for each aggregation in `agg`.

Groupby operations in Polars are fast because Polars has a parallel algorithm for getting the groupby keys. Aggregations are also fast because Polars runs multiple expressions in `agg` in parallel.

### Window operations
Window operations occur when we want to add a column that reflects not just data from that row but from a related group of rows. Windows occur in many contexts including rolling or temporal statistics and Polars covers these use cases.

Another example of a window operation is when we want on each row to have a statistic for a group of rows. We use the `over` expression for this (equivalent to `groupby-transform` in Pandas).

In this example we are going to add a column with the maximum age of the passenger in each class. To add a column we use an expression inside the `with_columns` method (we see much more of this method in Section 2). In the expression we calculate the maximum `Age` and specify that we want here we use `over` to calculate that max by the passenger class

In [ ]:
(
    df
    .with_columns(
        pl.col("Age").max().over("Pclass").alias("MaxAge")
    )
    .select("Pclass","Age","MaxAge")
    .head(3)
)

> We learn more about grouping and aggregations in Section 4 of the course.

### Visualisation

We can use popular plotting libraries like Matplotlib, Seaborn, Altair and Plotly directly with Polars.

In this example we create a scatter plot of bar chart of age and fare with Altair (version 5+ of Altair)

In [ ]:
import altair as alt
alt.Chart(
    df,
    title="Scatter plot of Age and Fare"
).mark_circle().encode(
    x="Age:Q",
    y="Fare:Q"
)

> We see how to work with Matplotlib, Seaborn, Altair and Plotly in the visualisation lecture in this Section.

## Lazy mode and query optimisation
In the examples above we work in eager mode. In eager mode Polars runs each part of a query step-by-step.

Polars has a powerful feature called lazy mode. In this mode Polars looks at a query as a whole to make a query graph. Before running the query Polars passes the query graph through its query optimiser to see if there ways to make the query faster.

When working with a CSV we can switch from eager mode to eager mode by replacing `read_csv` with `scan_csv`

In [ ]:
(
    pl.scan_csv(csv_file)
    .group_by(["Survived","Pclass"])
    .agg(
        pl.col("PassengerId").count().alias("count")
    )
)

The output of a lazy query is `LazyFrame` and we see the unoptimized query plan when we output a `LazyFrame`.

### Query optimiser
We can see the optimised query plan that Polars will actually run by add `explain` at the end of the query

In [ ]:
print(
    pl.scan_csv(csv_file)
    .group_by(["Survived","Pclass"])
    .agg(
        pl.col("PassengerId").count().alias("count")
    )
    .explain()
)

In this example Polars has identified an optimisation:
```python
PROJECT 3/12 COLUMNS
```
There are 12 columns in the CSV, but the query optimiser sees that only 3 of these columns are required for the query. When the query is evaluated Polars will `PROJECT` 3 out of 12 columns: Polars will only read the 3 required columns from the CSV. This projection saves memory and computation time.

A different optimisation happens when we apply a `filter` to a query. In this case we want the same analysis of survival by class but only for passengers over 50

In [ ]:
print(
    pl.scan_csv(csv_file)
    .filter(pl.col("Age") > 50)
    .group_by(["Survived","Pclass"])
    .agg(
        pl.col("PassengerId").count().alias("count")
    )
    .explain()
)

In this example the query optimiser has seen that:
- 4 out of 12 columns are now required `PROJECT 4/12 COLUMNS` and
- only passengers over 50 should be selected `FILTER: [(col("Age")) > (50.0)]`

These optimisations are applied as Polars reads the CSV file so the whole dataset must not be read into memory.

### Query evaluation

To evaluate the full query and output a `DataFrame` we call `collect` 

In [ ]:
(
    pl.scan_csv(csv_file)
    .filter(pl.col("Age") > 50)
    .group_by(["Survived","Pclass"])
    .agg(
        pl.col("PassengerId").count().alias("count")
    )
    .collect()
)

We learn more about lazy mode and evaluating queries in this section of the course.

## Streaming larger-than-memory datasets
By default Polars reads your full dataset into memory when evaluating a lazy query. However, if your dataset is too large to fit into memory Polars can run many operations in *streaming* mode. With streaming Polars processes your query in batches rather than all at once.

To enable streaming we pass the `streaming = True` argument to `collect`

In [ ]:
(
    pl.scan_csv(csv_file)
    .filter(pl.col("Age") > 50)
    .group_by(["Survived","Pclass"])
    .agg(
        pl.col("PassengerId").count().alias("count")
    )
    .collect(streaming = True)
)

In the course we look at what queries streaming can be used in (see the Streaming CSV lecture in the I/O section for more detail).

## Summary
This notebook has been a quick overview of the key ideas that make Polars a powerful data analysis tool:
- expressions allow us to write complex transformations concisely and run them in parallel
- lazy mode allows Polars apply query optimisations that reduce memory usage and computation time
- streaming lets us process larger-than-memory datasets with Polars